In [1]:
#file used to generate outputs for presentations
import warnings
#use this when printing the output to a pdf
warnings.filterwarnings('ignore')
#use this when doing coding normally
#warnings.filterwarnings('default')


In [35]:
import pandas as pd
import numpy as np
from sys import path
path.append("..")
from config import *

In [36]:
sae = pd.read_csv(DATA_FOLDER + "/sae.csv")
#cases = pd.read_csv("data/Cases.csv")
#hours = pd.read_csv("data/hours.csv")
cases = pd.read_csv(DATA_FOLDER + "/random_cases.csv")
hours = pd.read_csv(DATA_FOLDER + "/random_hours.csv")


Merging hours and cases

In [14]:
"""# Cases


* Change date to date_time object
    * Create a new Date column for both datasets
    * Allows merging on shared column
*  Sort dates in ascending order


"""

cases

# check type, length
cases.CLNDR_DT

#change CLNDR_DT from object to date_time
  # make sure to create new column 'Date' for both datasets so they share common 
  # for merging

cases['Date'] = pd.to_datetime(cases['CLNDR_DT'])
cases

#sort dates from past to current
cases = cases.sort_values(by='CLNDR_DT')
cases

#drop redundant columns

#^Dont need this line
#cases = cases.drop(columns=['CLNDR_DT', 'DIV_NM'])
#cases

# check number of NaN values
cases.isna().sum()

## needed to drop NaN in order for 'tot_each_day' to be correct number
    ## ask why including NaN would yield wrong sum
        ## what values does NaN have?

cases = cases.dropna()
cases

## create new dataframe from cases
    ## focusing on getting total of eaches selected per day
cases_tot_day = pd.DataFrame(cases)

## group by Date and BRNCH_CD, sum eaches_selected by each zone into one total each column
cases_tot_day['Total_Each_Day'] = pd.DataFrame(cases.groupby(['Date', 'BRNCH_CD'])['EACHES_SELECTED'].transform('sum'))
cases_tot_day

## New Dataframe with order = Date, Branch, zone, eaches selected, total eaches
cases4 = pd.DataFrame(cases, columns = (['Date', 'BRNCH_CD', 'ZONE', 'EACHES_SELECTED', 'Total_Each_Day']))

## Change index starting at 1, drop existing index to avoid adding as column
cases4 = cases4.reset_index(drop=True)

cases4.BRNCH_CD[0]

## get ratio of total eaches per zone
    ## eaches selected of zone / total eaches
    ## add ratio to a list and append to dataset after

ratio = []
for i in range(0, len(cases4)):
  #print(i)
  r = cases4.EACHES_SELECTED[i] / cases4.Total_Each_Day[i]
  ratio.append(r)

ratio[0:10]

## add list of ratios to dataframe, should be in order
cases4['ratio'] = ratio

#type(cases4.ZONE[1])

## create a list for every zone (dry, clr, frz)
## for every row in dataframe:
    ## if the zone of the row == "DRY":
        ## add the ratio at index i to the "DRY" list
        ## add zeros to the other lists (ClR, FRZ) (place holder)
    ## if zone of row == "CLR":
        ## add ratio to CLR list
        ## add zeros to other lists
    ## repeat for FRZ

## add lists as columns to dataset

dry_ratio = []
clr_ratio = []
frz_ratio = []

for i in range(0, len(cases4)): 
  if cases4.ZONE[i] == 'DRY':
    dry_ratio.append(cases4.ratio[i])
    clr_ratio.append(0)
    frz_ratio.append(0)
  elif cases4.ZONE[i] == 'FRZ':
    dry_ratio.append(0)
    clr_ratio.append(0)
    frz_ratio.append(cases4.ratio[i])
  elif cases4.ZONE[i] == 'CLR':
    dry_ratio.append(0)
    clr_ratio.append(cases4.ratio[i])
    frz_ratio.append(0)


cases4['dry_ratio'] = dry_ratio
cases4['clr_ratio'] = clr_ratio
cases4['frz_ratio'] = frz_ratio

## sort dataframe by date and branch to see everything together
    ## check if math is correct
cases4 = cases4.sort_values(by=['Date', 'BRNCH_CD'], ascending = [True, True])

## smoosh multiple rows of same branch and date into 1 row
    ## aggregate sums, zeros as placeholders will have no effect on ratio
cases6 = cases4.groupby(['Date', 'BRNCH_CD', 'Total_Each_Day'], as_index = False).agg('sum')


## drop eaches since same as total
## drop ratio since all ratios add up to 1 anyway
cases_EachPerDay = cases6.drop(columns=['EACHES_SELECTED', 'ratio'])

"""# Hours


*   Repeat steps from cases


"""

hours.REPORT_DATE

hours['Date'] = pd.to_datetime(hours['REPORT_DATE'])

hours = hours.sort_values(by=['REPORT_DATE', 'BRNCH_CD'], ascending = [True, True])

hours = hours.drop(columns=['DIV_NBR', 'REPORT_DATE', 'FULL_MARKET_NAME'])

## drop "NaN" (UNKOWN)
hours = hours[hours.COHORT != 'UNKNOWN']

## create datarame in order desired
hours_co = pd.DataFrame(hours, columns = ['Date', 'BRNCH_CD', 'COHORT', 'CASES_SELECTED', 'MEASURED_DIRECT_HRS'])

## group by Date and branch to get sum of HOURS MEASURED of each cohort as total hours worked at branch on specific day
hours_co['Total_Hours'] = pd.DataFrame(hours.groupby(['Date', 'BRNCH_CD'])['MEASURED_DIRECT_HRS'].transform('sum'))


## group by Date and branch to get sum of CASES SELECTED of each cohort as total cases selected at branch on specific day
hours_co['Total_Cases'] = pd.DataFrame(hours.groupby(['Date', 'BRNCH_CD'])['CASES_SELECTED'].transform('sum'))


hours_co = hours_co.reset_index(drop = True)

## like hours, get ratio of hours worked per cohort relative to total hours measured at branch on specific day
Hrs_PerCo = []
for i in range (0, len(hours_co)):
  r = hours_co.MEASURED_DIRECT_HRS[i] / hours_co.Total_Hours[i]
  Hrs_PerCo.append(r)

Hrs_PerCo[0:5]

hours_co['Hrs_Pct'] = Hrs_PerCo
hours_co

## same process of placing respective hours ratios per cohort as eaches per zone in cases
    ## 0.000 = placeholder for aggregation
A_Hours = []
B_Hours = []
C_Hours = []

for i in range(0, len(hours_co)):
  if hours_co.COHORT[i] == 'A':
    A_Hours.append(hours_co.Hrs_Pct[i])
    B_Hours.append(0)
    C_Hours.append(0)
  elif hours_co.COHORT[i] == 'C':
    A_Hours.append(0)
    B_Hours.append(0)
    C_Hours.append(hours_co.Hrs_Pct[i])
  elif hours_co.COHORT[i] == 'B':
    A_Hours.append(0)
    B_Hours.append(hours_co.Hrs_Pct[i])
    C_Hours.append(0)

hours_co['A_HrsPct'] = A_Hours
hours_co['B_HrsPct'] = B_Hours
hours_co['C_HrsPct'] = C_Hours

## ratio of cases per cohort
Cases_PerCo = []
for i in range (0, len(hours_co)):
  r = hours_co.CASES_SELECTED[i] / hours_co.Total_Cases[i]
  Cases_PerCo.append(r)

hours_co['Cases_Pct'] = Cases_PerCo

## cases ratio per cohort w/ placeholders
A_Cases = []
B_Cases = []
C_Cases = []

for i in range(0, len(hours_co)):
  if hours_co.COHORT[i] == 'A':
    A_Cases.append(hours_co.Cases_Pct[i])
    B_Cases.append(0)
    C_Cases.append(0)
  elif hours_co.COHORT[i] == 'C':
    A_Cases.append(0)
    B_Cases.append(0)
    C_Cases.append(hours_co.Cases_Pct[i])
  elif hours_co.COHORT[i] == 'B':
    A_Cases.append(0)
    B_Cases.append(hours_co.Cases_Pct[i])
    C_Cases.append(0)


hours_co['A_Cases'] = A_Cases
hours_co['B_Cases'] = B_Cases
hours_co['C_Cases'] = C_Cases

hours_co

hours_CoHrs = hours_co.groupby(['Date', 'BRNCH_CD', 'Total_Hours', 'Total_Cases'], as_index = False).agg('sum')
hours_CoHrs

"""# New Datasets"""

cases_EachPerDay

## Date = CLNDR_DATE
## BRNCH_CD = code referring to sepcific Warehouse

## Total_Each_Day = sum of eaches selected by all zones on specific date
    ## dry_ratio = proportion of dry products relative to total eaches selected on specific date 
    ## clr_ratio = proportion of cooler products relative to total eaches selected on specific date
    ## frz_ratio = proportion of freezer products relative to total eaches selected on specific date

hours_CohortHrsCases = pd.DataFrame(hours_CoHrs, columns = ['Date', 'BRNCH_CD', 
                                                            'Total_Hours', 'Total_Cases',
                                                            'A_HrsPct',
                                                            'B_HrsPct', 'C_HrsPct', 
                                                            'A_Cases', 'B_Cases','C_Cases' 
                                                            ])
hours_CohortHrsCases

## Date = REPORT_DATE
## BRNCH_CD = code referring to sepcific Warehouse

## Total_Hours = sum of hours worked by all cohorts on specific date
    ## A_HrsPct = proportion of hours worked by Cohort A relative to total hours on specific date
    ## B_HrsPct = proportion of hours worked by Cohort B relative to total hours on specific date
    ## C_HrsPct = proportion of hours worked by Cohort C relative to total hours on specific date

## Total_Cases = Sum of all cases selected by all cohorts on specific date
    ## A_Cases = proportion of cases selected by Cohort A relative to total cases selected on specific date
    ## B_Cases = proportion of cases selected by Cohort B relative to total cases selected on specific date
    ## C_Cases = proportion of cases selected by Cohort C relative to total cases selected on specific date

"""# Merge
* lost a lot of data :/
* dont merge yet?
"""

Foods = hours_CohortHrsCases.merge(cases_EachPerDay, how = 'inner', on = ['Date', 'BRNCH_CD'])


FileNotFoundError: [Errno 2] No such file or directory: 'src/datasae.csv'

Adding dates to the dataset and merging merged dataset with the sae dataset. Also created dummy variables for the categorical variables (month, weekday, branch_cd)

In [3]:
Foods2 = Foods

#adding the response variable(s) to the dataset
Foods2["cases_hrs"] = Foods2['Total_Cases']/Foods2['Total_Hours'] #creates the dependent variable we are trying to measure
Foods2.replace([np.inf, -np.inf], np.nan, inplace=True) #converting all infinites to na's. will want to come up with a better solution later 
Foods2 = Foods2.dropna() #drops all na's for now

weekdays = {0: "Monday",
            1: "Tuesday",
            2: "Wednesday",
            3: "Thursday",
            4: "Friday",
            5: "Saturday",
            6: "Sunday"}

months =   {1: "January",
            2: "February",
            3: "March",
            4: "April",
            5: "May",
            6: "June",
            7: "July",
            8: "August",
            9: "September",
            10: "October",
            11: "November",
            12: "December"}
            
Foods2['weekday'] = Foods2['Date'].apply(lambda x: weekdays[x.weekday()]) #adds the weekday of the particular entry 
Foods2['month'] = Foods2['Date'].apply(lambda x: months[x.month]) #add the reporting_date month as a separate variable


#adds the sae data to the main dataset
sae = pd.read_csv(DATA_FOLDER + '/sae.csv')

Foods3 = Foods2.merge(sae, how='outer', on='BRNCH_CD')
Foods3.to_csv(DATA_FOLDER + '/master_dataset.csv', index=False)

#Foods3.head()